In [24]:
import numpy as np
from math import *
from numpy.linalg import norm

In [25]:
def osc2eqx(y):
    #y = a,  e,  i,  Ω,  ω, M0
    #z = p, ex, ey, ix, iy,  L
    #    0,  1,  2,  3,  4,  5
    z = np.empty((6,))
    z[0] = y[0] * (1 - y[1]**2)
    z[1] = y[1] * cos(y[4] + y[3])
    z[2] = y[1] * sin(y[4] + y[3])
    z[3] = tan(y[2] / 2) * cos(y[3])
    z[4] = tan(y[2] / 2) * sin(y[3])
    M = y[5] + sqrt(mu / y[0]**3) * (t - t0)
    z[5] = 1 # Здесь должны быть ещё уравнения

In [26]:
def vector_mm(vector):
    A = np.zeros((3, 3))
    A[0, 1] = -vector[2]
    A[0, 2] = vector[1]
    A[1, 2] = -vector[0]
    A = A - A.T
    return A

In [27]:
def integral_h(r, v):
    return norm(v)**2 - 2*mu / norm(r)

In [28]:
def integral_c(r, v):
    c = vector_mm(r) @ v
    c0 = norm(c)
    return (c, c0)

In [29]:
def integral_A(r, v):
    c = integral_c(r, v)[0]
    return vector_mm(v) @ c - mu * r / norm(r) 

In [30]:
def eqx2rv(z):
    #z = p, ex, ey, ix, iy, L
    #    0,  1,  2,  3,  4, 5
    f = sqrt(mu * z[0]) / (1 + z[3]**2 + z[4]**2) * np.array([1 + z[3]**2 - z[4]**2, 2*z[3]*z[4], -2*z[4]])
    g = sqrt(mu * z[0]) / (1 + z[3]**2 + z[4]**2) * np.array([2*z[3]*z[4], 1 - z[3]**2 + z[4]**2,  2*z[3]])
    rvect = sqrt(z[0]/mu) * (f * cos(z[5]) + g * sin(z[5])) / (1 + z[1]*cos(z[5]) + z[2]*sin(z[5]))
    vvect = (z[1] + cos(z[5])) / z[0] * g - (z[2] + sin(z[5])) / z[0] * f
    return np.append(rvect, vvect)

In [39]:
def rv2eqx(rv):
    #z = p, ex, ey, ix, iy, L
    #    0,  1,  2,  3,  4, 5
    z = np.empty((6,))
    rvect = rv[0:3].copy()
    vvect = rv[3:6].copy()
    A = integral_A(rvect, vvect)
    c, c0 = integral_c(rvect, vvect)
    f = np.array([c0 - c[0]**2 / (c0 + c[2]), -c[0]*c[1] / (c0 + c[2]), -c[0]])
    g = np.array([-c[0]*c[1] / (c0 + c[2]), c0 - c[1]**2 / (c0 + c[2]), -c[1]])
    z[0] = c0**2 / mu
    z[1] = np.dot(A, f) / c0 / mu
    z[2] = np.dot(A, g) / c0 / mu
    z[3] = -c[1]/(c0 + c[2])
    z[4] = c[0]/(c0 + c[2])
    z[5] = atan2(np.dot(rvect, g), np.dot(rvect, f))
    return z

In [40]:
mu = 1

In [41]:
r = np.array([1.0, 0.0, 0.0])
v = np.array([0.0, 1.0, 0.0])
print(eqx2rv(rv2eqx(np.append(r, v))))

[ 1.  0. -0. -0.  1.  0.]


In [52]:
k = 10
for i in range(10000):
    x = np.empty((6,))
    for j in range(6):
        x[j] = k * (1 - 2 * np.random.random())
    bar = norm(x - eqx2rv(rv2eqx(x)))
    if bar > 1e-12:
        print(bar)

2.0237926356358747e-12
1.8399423693986424e-12
2.838035515776108e-12
